# ClimApp Translations: CSV to JSON 
----
This jupyter nootebook is used to convert three CSV files to one JSON objects:
the files need to follow the below naming scheme:

#### Input
* Toasts: `climapp_translation_sheet_toasts.csv`
* Wheels: `climapp_translation_sheet_wheels.csv` 
* Text: `climapp_translation_sheet_text.csv`

#### Output
* `combined_object`: `translations.json`


#### Procedure
Each file is read one by one, converted in different ways and merged into one JSON object that will be used in the ClimApp mobile application for translating the app content.

**The array `availableLanguages` contains an overview of all current languages. Update this if more languages are made available. **

----

#### Converting notebook to executable
https://stackoverflow.com/questions/55741607/is-it-possible-to-generate-an-executable-exe-in-a-jupyter-notebook

## Code
**Importing needed libraries**

In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
os.getcwd()

'C:\\Users\\frksteenhoff\\Documents\\GitHub\\Smaller projects'

## Functions

Definitions of the two functions used for conversion

* `toast` and `text` use **convertCSVtoJSON_simple**
* `wheels` use **convertCSVtoJSON_nested**

In [7]:
# Converting simple CSV structure to json
# df                 - dataframe with csv content
# availableLanguages - languages to translate
def convertCSVtoJSON_simple(df, availableLanguages):
    json = {}
    for i in range(len(df['text_id'])):
        json[df['text_id'][i]] = { lang : df[lang][i] for lang in availableLanguages}
    return json

In [8]:
# Converting CSV content to nested json objet
# df                 - dataframe with csv content
# availableLanguages - languages to translate
def convertCSVtoJSON_nested(df, availableLanguages):
    wheels = {}
    # Add all unique text_id keys
    for val in df['text_id'].unique():
        wheels[val] = {}

    # For all nested keys for each text_id if not exist, add
    for i in range(len(df['key2'])):

        if(df['key2'][i] not in wheels[df['text_id'][i]]):        
            # Add translations for all available languages
            wheels[df['text_id'][i]].update({df['key2'][i] : {}})

        if(pd.isnull(df['key3'][i])):
            # If key3 is null, add translations directly
            wheels[df['text_id'][i]][df['key2'][i]].update({lang : df[lang][i] for lang in availableLanguages})
        else:
            # If key3 has a value, add languages as childs of the key3 value
            wheels[df['text_id'][i]][df['key2'][i]].update({df['key3'][i] : {lang : df[lang][i] for lang in availableLanguages}})
    return wheels

----
## Text

#### Reading in file with general strings

In [4]:
df = pd.read_csv("climapp_translation_sheet - wheels.csv")
df.head()

,text_id,key2,key3,en,da,nl,sv,no,el,it,de,es,fr
0,activity,title,NaN,What is your activity level?,Hvad er dit aktivitetsniveau?,NaN,NaN,Hva er aktivitetsnivået ditt?,NaN,Qual è il tuo livello di attività?,Was ist dein Aktivitätsniveau,NaN,Quel est votre niveau d'activité ?
1,activity,description,rest,"Resting, sitting at ease.\nBreathing not chall...","Hviler, sidder roligt. Vejrtrækning normal",NaN,NaN,"Hviler, sitter i ro. \nPusten er ikke forstyrret.",NaN,"Riposarsi, rimanere seduti.\n Respirazione nor...","Ruhe, still sitzen. \nAtmen nicht herausgefordert",NaN,"Se reposer, s'asseoir à l'aise. \nRespiration ..."
2,activity,description,light,"Light manual work:\nwriting, typing, drawing, ...","Let manuelt arbejde: skrive, taste, tegne, bog...",NaN,NaN,"Lett manulet arbeid: skriving, tegning, bokfør...",NaN,"Leggero lavoro manuale: scrittura, disegno, co...","Leichte manuelle Arbeit:\nschreiben, zeichnen,...",NaN,"Travail manuel léger : écrire, taper à l'ordin..."
3,activity,description,medium,Walking 2.5 - 5.5km/h. Sustained arm and hand ...,Gang 2.5-5.5 km/t. Vedvarende arm og håndarbej...,NaN,NaN,Gå 2.5-5.5 km/t. Vedvarende arm- og håndarbeid...,NaN,"Camminata a 2,5-5,5 km / h. Lavori di braccia ...","Gehen 2,5 - 5,5 km/h. Andauernd Arm- und Handa...",NaN,"Marcher à 2,5 - 5,5km/h. Travail soutenu des b..."
4,activity,description,high,Intense arm and trunk work: carrying heavy mat...,"Intens aktivitet: bære tungt materiale, skovle...",NaN,NaN,"Intens armarbeid: bæring av tungt materiale, m...",NaN,Intenso lavoro di braccia e tronco: trasporto ...,Intensive Armarbeit: Tragen schwerer Materiali...,NaN,Travail intense des bras et du tronc : porter ...


#### **Available languages**
NOTE: This information is used in all conversions, translations MUST be completed in the same languages for `text`, `toasts` and `wheels` for the program to work correctly.

* all columns but the first contains languages, we only include those that have values for all keys.
* If there are ANY null values within the translations, the language will be escluded. 
* It is important only to include rows with values!



In [5]:
# Extracting available languages from the text sheet

availableLanguages = []

print("All languages in sheet: ", df.columns[1:].values, "\n")

for lang in df.columns[1:]:
    if(not pd.isnull(df[lang]).any()):
        print("'" + lang + "' translations are added correctly")
        availableLanguages.append(lang)
    else:
        print("'" + lang + "' translations are MISSING VALUES")

print("\nCurrently available languages: ", availableLanguages)

All languages in sheet:  ['key2' 'key3' 'en' 'da' 'nl' 'sv' 'no' 'el' 'it' 'de' 'es' 'fr'] 

'key2' translations are added correctly
'key3' translations are MISSING VALUES
'en' translations are added correctly
'da' translations are added correctly
'nl' translations are MISSING VALUES
'sv' translations are MISSING VALUES
'no' translations are added correctly
'el' translations are MISSING VALUES
'it' translations are added correctly
'de' translations are added correctly
'es' translations are MISSING VALUES
'fr' translations are added correctly

Currently available languages:  ['key2', 'en', 'da', 'no', 'it', 'de', 'fr']


#### Converting csv content to JSON object

In [9]:
text = convertCSVtoJSON_simple(df, availableLanguages)

In [10]:
print(json.dumps(text, indent=4))

{
    "activity": {
        "key2": "label",
        "en": "Intense",
        "da": "Intens",
        "no": "Intens",
        "it": "Intenso",
        "de": "Intensiv",
        "fr": "Intense"
    },
    "clothing": {
        "key2": "label",
        "en": "Winter attire",
        "da": "Vintert\u00f8j ",
        "no": "Vinterkl\u00e6r",
        "it": "Abbigliamento invernale",
        "de": "Winterkleidung",
        "fr": "Tenue d'hiver"
    },
    "settings": {
        "key2": "_humidity",
        "en": "Input the humidity you would like to use",
        "da": "Angiv den luftfugtighed du kunne t\u00e6nke dig at benytte",
        "no": "Spesifiser den luftfuktigheten du \u00f8nsker \u00e5 bruke",
        "it": "Specificare l'umidit\u00e0 che si desidera utilizzare",
        "de": "Geben Sie die Luftfeuchtigkeit ein, die Sie verwenden wollen",
        "fr": "Entrer l'humidit\u00e9 que vous souhaitez utiliser"
    },
    "headgear": {
        "key2": "label",
        "en": "Helmet or ho

-----
## Toasts

#### Reading in file with toast text

In [367]:
df = pd.read_csv("C:\\Users\\frksteenhoff\\Google Drive\\DTU - Master\\ClimApp\\climapp_translation_sheet_toasts.csv")

**Showing basic content**

In [368]:
df.head()

,key1,text_id,en,da,nl,sv,no
0,toasts,toast__error,Whoops... something went wrong at:,Uups.. noget gik galt ved:,NaN,NaN,NaN
1,toasts,toast__toast_adv_user,After 5 uses you are now considered an experie...,"Efter at du nu har brugt appen 5 gange, anses ...",NaN,NaN,NaN
2,toasts,toast__toast_using_default,Using default values in calculations.,Anvender standard værdier i beregninger,NaN,NaN,NaN
3,toasts,toast_fb_adaptation_reset,Adaptation level reset to 0,Tilpasningsniveau nustillet,NaN,NaN,NaN
4,toasts,toast_sett_preferences_reset,Personal preferences reset,Personlige preferencer nulstillet,NaN,NaN,NaN


#### Converting csv content to JSON object

In [369]:
toasts = convertCSVtoJSON_simple(df, availableLanguages)

In [374]:
print(json.dumps(toasts, indent=4))

{
    "toast__error": {
        "en": "Whoops... something went wrong at: ",
        "da": "Uups.. noget gik galt ved:"
    },
    "toast__toast_adv_user": {
        "en": "After 5 uses you are now considered an experienced user.",
        "da": "Efter at du nu har brugt appen 5 gange, anses du for at v\u00e6re en erfaren bruger."
    },
    "toast__toast_using_default": {
        "en": "Using default values in calculations.",
        "da": "Anvender standard v\u00e6rdier i beregninger"
    },
    "toast_fb_adaptation_reset": {
        "en": "Adaptation level reset to 0",
        "da": "Tilpasningsniveau nustillet"
    },
    "toast_sett_preferences_reset": {
        "en": "Personal preferences reset",
        "da": "Personlige preferencer nulstillet"
    },
    "toast_sett_acclimatized": {
        "en": "You are acclimatized to your working environment.",
        "da": "Du er akklimatiseret til dit arbejdsmilj\u00f8"
    },
    "toast_sett_not_acclimatized": {
        "en": "You are n

----
## Wheels

#### Reading in file with wheels text

In [364]:
df = pd.read_csv("C:\\Users\\frksteenhoff\\Google Drive\\DTU - Master\\ClimApp\\climapp_translation_sheet_wheels.csv")
df.head()

,text_id,key2,key3,en,da,nl,sv,no
0,activity,title,NaN,What is your activity level?,Hvad er dit aktivitetsniveau?,NaN,NaN,NaN
1,activity,description,rest,"Resting, sitting at ease.\nBreathing not chall...","Hviler, sidder i ro. Vejrtrækning ikke udfordret",NaN,NaN,NaN
2,activity,description,light,"Light manual work:\nwriting, typing, drawing, ...","Let manuelt arbejde: skrive, taste, tegne, bog...",NaN,NaN,NaN
3,activity,description,medium,Walking 2.5 - 5.5km/h. Sustained arm and hand ...,Bevægelse 2.5-5.5 km/t. Vedvarende arm og hånd...,NaN,NaN,NaN
4,activity,description,high,Intense arm and trunk work: carrying heavy mat...,"Intens aktivitet: bære tungt materiale, skovle...",NaN,NaN,NaN


#### Converting csv content to JSON object

In [365]:
wheels = convertCSVtoJSON_nested(df, availableLanguages)

In [366]:
# Use to check the result
print(json.dumps(wheels, indent=4))

{
    "activity": {
        "title": {
            "en": "What is your activity level?",
            "da": "Hvad er dit aktivitetsniveau?"
        },
        "description": {
            "rest": {
                "en": "Resting, sitting at ease.\\nBreathing not challenged.",
                "da": "Hviler, sidder i ro. Vejrtr\u00e6kning ikke udfordret"
            },
            "light": {
                "en": "Light manual work:\\nwriting, typing, drawing, book-keeping.\\nEasy to breathe and carry on a conversation.",
                "da": "Let manuelt arbejde: skrive, taste, tegne, bogf\u00f8ring. Nemt at tr\u00e6kke vejret og f\u00f8re en samtale"
            },
            "medium": {
                "en": "Walking 2.5 - 5.5km/h. Sustained arm and hand work: handling moderately heavy machinery, weeding, picking fruits.",
                "da": "Bev\u00e6gelse 2.5-5.5 km/t. Vedvarende arm og h\u00e5ndarbejde: h\u00e5ndtering af moderat tungt maskineri, plukning af ukrudt eller frugte

## Combining all information in one JSON object

In [297]:
combined_object = {"text" : text, "wheels" : wheels, "toasts" :toasts}

In [298]:
print(json.dumps(combined_object, indent=4))

{
    "text": {
        "str_dashboard": {
            "en": "Dashboard",
            "da": "Dashboard"
        },
        "str_settings": {
            "en": "Settings",
            "da": "Indstillinger"
        },
        "str_humidity": {
            "en": "Humidity",
            "da": "Luftfugtighed"
        },
        "str_male": {
            "en": "Male",
            "da": "Mand"
        },
        "str_female": {
            "en": "Female",
            "da": "Kvinde"
        }
    },
    "wheels": {
        "activity": {
            "title": {
                "en": "What is your activity level?",
                "da": "Hvad er dit aktivitetsniveau?"
            },
            "description": {
                "rest": {
                    "en": "Resting, sitting at ease.\\nBreathing not challenged.",
                    "da": "Hviler, sidder i ro. Vejrtr\u00e6kning ikke udfordret"
                },
                "light": {
                    "en": "Light manual work:\\nwriti

### Saving the above struture to a .JSON file

In [299]:
with open('C:\\Users\\frksteenhoff\\Google Drive\\DTU - Master\\ClimApp\\translations.json', 'w') as f:
    json.dump(combined_object, f)